# Ipylab

`Ipylab` is the primary class that provides functionality for scheduling and awaiting operations between Python objects and the corresponding model in the frontend. Each operation is run as a Task which is assigned a UUID and sent as a custom message. 

`IpylabModel` has a `base` object that is set prior to being *ready*. The base object is specified by the Python class and located by the frontend model with the `ipylabInit` method (customised by other subclasses). 

## Operations

The `operation` method is the workhorse through which all asynchronous messaging is done. It will always return a task. Each operation much be implement in the frontend model. 


## Generic methods

 `Ipylab` implements the following generic methods interact with objects in the fronted: 

* execute_method
* get_property
* set_property
* update_property
* list_properties

These methods pass an object to utility functions reducing the need to write corresponding code in the frontend. 

By default the methods the `base`. A different base object can be specified `obj=Obj.###` to specify a different path.

### Signals

* list_signals
* list_view_signals
* register_signal_callback

These methods provide for identification of signals and registering callbacks to those signals in the frontend.


## Connection

A `Connection` is subclassed from `Ipylab` providing a connection to an object in the frontend. 

Each connection has a unique `connection_id`.  The `connection_id` consists of the class prefix `ipylab-<CLASS NAME>|` followed by one or more parts joined with a pipe. The class prefix part of the `connection_id` determines the type of class that is created when creating a new instance.

In the kernel there is only one instance of a Connection object per `connection_id`. For example: calling `Connection('ipylab-Connection|my connection_id')` multiple times will get the same object inside the same kernel.

The object in the frontend is stored as an ObservableDisposable and once it has be set, it cannot be replaced with another object, unless it has been disposed.

Calling the close method in Python will by default dispose of the object in the frontend. This can behavior can be changed by using `dispose=False`, or previously setting the 'auto_dispose` trait to False.


### Subclasses

Subclasses of `Connection` are identified by its *prefix*. Connection subclasses provide specific functionality relevant to the object. Generally, connections are stored in the `connection` attributed of the class in which they were created.

### Connection models

Currently there are two Javascript classes models for connections, `ConnectionModel` and `ShellConnectionModel`. `ShellConnectionModel` is specific for widgets that are loaded in the shell.

### Making a connection

Normally a connection is not instantated directly. To obtain a connection specify the `transform` as a `Transform.connection`.

#### Examples:

* [Autostart](plugins.ipynb#Example-launching-a-small-app)
* [Console](commands.ipynb#Create-a-new-console)
* [Signals - connection](#Connection)

In [ ]:
import ipylab

app = await ipylab.App().ready()

await app.list_properties("shell", depth=2)

Some of the methods above have already been implemented in 'app.shell' on the Python object. And if you look at the code you'll see these methods use the *generic* method `execute_method`.

In case you're interested, the custom message sent to the frontend looks like this:

```python
{
    "ipylab": '{"ipylab_PY": "9b22d58e-e48a-4d8c-84ad-2f92e1b5783e", "operation": "genericOperation", "kwgs": {"depth": 3, "omitHidden": true, "genericOperation": "listProperties", "basename": "base", "subpath": ""}, "transform": "auto"}'
}
```

Note that *ipylab* is always JSON string. This is because we convert every message to JSON prior to sending making it possible to replace widgets and code objects with string representations. For widgets, it is necessary to instruct the frontend what to do with the widget. Currently there are the following additional options:
1. `toLuminoWidget`: Replace with a widget in the frontend.
2. `toObject`: Replace with an object in the frontend.

These options are lists of attributes as they appear in the frontend that should be transformed/extracted.

In [ ]:
# The source code

import inspect

print(inspect.getsource(app.shell.expand_left))

In [ ]:
await app.shell.expand_right()  # Built in

In [ ]:
await app.shell.execute_method("collapseRight")  # Generic

Note that all Python methods are written with `snake_case` whereas the Javascript command is written as `cammelCase`.

## Signals

The `Ipylab` class provides methods to list the signals on their corresponding base object in the frontend. If the model also has a view, such as `SimpleOutput` and `CodeEditor` it is also possible to list and register callbacks to signals in the views. The depth parameter specifies the level on introspection on the Javascript object. Using a higher number will search deaper in the class inheritance tree.

In [ ]:
import ipylab

app = ipylab.App()

# Signals for the Jupyterfrontend
await app.list_signals(depth=4)

In [ ]:
async def on_context_opened(_):
    await app.notification.notify("Context menu opened", type=ipylab.NotificationType.info, auto_close=True)


app.register_signal_callback("contextMenu.opened", on_context_opened)

You can now open the Context Menu using **right click**

In [ ]:
# Remove the callback
app.register_signal_callback("contextMenu.opened", on_context_opened, remove=True)

### Connection

We can connect to signals via the connection. In this case we will connect to this notebooks widget, and then connection to the 'content' of the widget.

In [ ]:
nb = await app.shell.connect_to_widget(app.shell.current_widget_id)
nb_content = await nb.get_property("content", transform=ipylab.Transform.connection)
await nb_content.list_signals(depth=3)

In [ ]:
out = ipylab.SimpleOutput()


def on_signal(data):
    out.push(data["args"])


nb_content.register_signal_callback("activeCellChanged", on_signal)
out

Here we get some content as the notebook is being navigated.

In [ ]:
# Disconnect
nb_content.register_signal_callback("activeCellChanged", on_signal, remove=True)

### CodeEditor

This signal is useful for providing history data as illustrated in the [Simple console](simple_output.ipynb#Simple-console-example) example.

In [ ]:
from ipylab import CodeEditor, SimpleOutput

In [ ]:
ce = await CodeEditor().ready()
display(ce)
# View signals
await ce.list_view_signals()

### SimpleOutput

In [ ]:
so = await SimpleOutput().ready()
display(so)
# View signals
await so.list_view_signals()